<img align = "left" width = 20% src="https://raw.github.com/paraguz1/solicitudes-dashboard/master/posts/Segurcaixa Adeslas.png">
        


<h1>Flujo solicitudes de baja canal asegurador</h1>

Solicitudes de baja Canal asegurador. Privados individuales, Pymes y Colectivos abiertos (de menos de 50?)


<img align = "left" width="80%" src="https://raw.github.com/paraguz1/solicitudes-dashboard/master/posts/Esquema Informe BajasRetención v6.jpg">



<h2> Índice</h2>

- [Flujo grabación y recuperación de solicitudes de baja](#1)
- [Solicitudes de baja flujo interactivo](#2) 


In [1]:
import plotly.graph_objects as go
import numpy as np
import collections
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from ipywidgets import interactive, HBox, VBox, widgets, Layout, IntSlider
from ipywidgets.embed import embed_minimal_html, embed_data
import json
import ipyvuetify as v
from IPython.display import clear_output


In [2]:
labels = ['Desconocido', 'Contact center', 'Oficinas Propias', 'OACs', 'Agentes Red', 'Corredores', 'Clínicas',
           'Siebel',  'ECO', 'EGAS', 'SOL', 'Cont útiles nivel 1', 'Cont no útiles nivel 1', 'Cola ECO/SOL', 'Cola EGAS', 'Contactos Útiles', 'Contactos No Útiles',
          'Retenidos', 'Bajas', 'Pend + Argum']

labels_index = list(range(len(labels)))

labels_dict = dict(zip (labels, labels_index))

#print (labels_dict)


labels2 = ['Bajas grabadas a recuperar', 'Solicitudes', 'Sin solicitud', 'Contactos útiles', 'Contactos no útiles','Retenidos',
           'Bajas', 'Pend + Argum']

labels2_index = list(range(len(labels2)))

labels2_dict = dict(zip (labels2, labels2_index))
#print ("")
#print (labels2_dict)



In [3]:
source    = [0,    1,     2  ,  2   , 2,    3 ,  3   , 3   ,   4,   4    , 4   ,  5   ,6  ,  0  , 7   ,   7   , 7 ,  11   ,11 , 11,   12   ,  8     , 8    ,8 ,  9   , 9 ,    9 , 9,   10, 10, 10,  13 ,  13,    14 ,    14  , 15  , 15,  15, 16]

target    = [18,   7,     7  ,  8   , 9,    1 ,  8   , 9   ,   1,   8    , 9   ,  1  , 1  , 10  , 11   ,  12  , 19,  17  , 19 , 18,   18   ,  13   ,  18,   19,  14,   18   , 17, 19,  13, 18, 19 , 15,   16,    15  ,   16  , 17,   18,   19, 18]
value     = [970, 18500, 3000, 1500,  1500, 500, 2200, 4000,  200,  3300 ,8000,  200, 100 , 30  , 13975,  7455, 70,  4000, 891, 9084, 7455 ,  2220  , 4770 ,10,  3850, 5910, 3700,40,  5 , 15, 5 ,  665,  1560,  1348,  2502,  428,  1587, 3, 4062]

#Custom data para informar los retenidos de cada casilla
#            0     1      2     3     4     5 ,   6    ,  7      8    9    10     11      12       13     14    15     16 17
cust_data = [0,    4891,  1200, 1215, 1716, 50  , 25   , 4891 ,  0, 3700 , 0    , 134  ,  297  ,   431 ,  0  ,  8128,  0, 894  ]



source2    = [0,     0,   1,    1,     1,   3,   3,    3,  4]

target2    = [1,     2,   3,    4,     7,   5,   6,    7,  6]
value2     = [21500, 350, 4300, 17100, 100, 471, 3827, 2,  17100 ]

#Custom data para informar los retenidos de cada casilla
#             0     1      2     3     4     5 ,   6    ,  7   
cust_data2 = [473,  473,   0,    473,  0,    473,  0,  2]




In [4]:
source_label = [ labels[index] for index in source ]

node_source_label = list(zip(source_label, source))
#print ('Source index-label list')
#print (' ')
#node_source_label

In [5]:
source_label2 = [ labels2[index] for index in source2 ]

node_source_label2 = list(zip(source_label2, source2))
#print ('Source index-label list')
#print (' ')
#node_source_label2

In [6]:
target_label = [ labels[index] for index in target ]

node_target_label = list(zip(target_label, target))
#print ('Target index-label list')
#print (' ')
#node_target_label

In [7]:
target_label2 = [ labels2[index] for index in target2 ]

node_target_label2 = list(zip(target_label2, target2))
#print ('Target index-label list')
#print (' ')
#node_target_label2

In [8]:
node_source_target_value = list(zip(source_label, target_label, value))
#print ('Source - Target value list')
#print (' ')
#node_source_target_value

In [9]:
node_source_target_value2 = list(zip(source_label2, target_label2, value2))
#print ('Source - Target value list')
#print (' ')
#node_source_target_value2

In [10]:
node_colors = [ 'rgba(255,255,0.7)',     #Desconocido amarillo 0
                'rgba(0,0,255,0.7)',     #Contact center, azul 1 
                'rgba(128,0,0,0.7)',     #Oficinas propias, granate 2
                'rgba(128,128,0,0.7)',   #Oacs, amarillo marrón 3
                'rgba(128,0,128,0.7)',   #Red comercial lila, 4
                
                'rgba(0,0,255,0.5)',    #Corredores azul claro 5
                'rgba(0,0,255,0.3)' ,   #Clínicas azul muy claro 6
                        
                'rgba(0,0,128, 0.7)',    #Siebel AZUL 7
                'rgba(0,0,128, 0.7)' ,   #ECO AZUL 8 
                'rgba(0,0,128, 0.7)',    #EGAS AZUL 9
                'rgba(0,0,128, 0.7)',     #SOL AZUL 10
               
                 'rgba(249,191,59, 1)' ,  #Contactos Útiles nivel 1 naranja claro 11
                'rgba(249,105,14, 1)',   #Contactos no Útiles  nivel 1naranja 12
               
               
                'rgba(0,128,128,0.6)' , #Sol Cola ECO aguamarina 13
                'rgba(0,128,128, 0.6)', #Sol Cola EGAS  aguamarina  14     
          
                'rgba(249,191,59, 1)' ,  #Contactos Útiles naranja claro 15
                'rgba(249,105,14, 1)',  #Contactos no Útiles naranja 16
              
                 'rgba(58,174,89, 1)' , #Retenidos verde 17
                 'rgba(255,0,0, 0.8)',  #No retenidos rojo 18 
                 'rgba(192,192, 192, 0.9)']  #Pendientes o disuadidas gris 19
                                

In [11]:
node_colors2 = ['rgba(255,0,0, 0.5)', #Bajas grabadas a recuperar rojo
                
                'rgba(0,0,128, 0.7)', #Solicitudes azul
                'rgba(255,0,0, 0.5)' , #Sin solicitud rojo claro
                
                'rgba(249,191,59, 1)' , #Contactos útiles naranja claro
                'rgba(249,105,14, 1)', #Contactos no útiles naranja
                
                'rgba(58,174,89, 1)' , #Retenidos verde
                'rgba(255,0,0, 0.8)',  #No retenidos rojo
                'rgba(192,192, 192, 0.9)']  #Pendientes o disuadidas gris   

In [12]:
flow_colors = ['rgba(255,0,0, 0.3)', # 'Desc no retenido'   0
                                      
               'rgba(0,0,128, 0.3)', #Contact center a Siebel  1
               'rgba(0,0,128, 0.3)', #Oficinas propias a Siebel 2
               'rgba(0,0,128, 0.3)', #Oficinas propias a ECO 3
               'rgba(0,0,128, 0.3)', #Oficnas propias a EGAS 4
               'rgba(0,0,128, 0.3)', #OACs a Contact center 5
               'rgba(0,0,128, 0.3)', #OACs a ECO  6
               'rgba(0,0,128, 0.3)', #OACs a EGAS 7
               'rgba(0,0,128, 0.3)', #Red agentes a Contact center 8
               'rgba(0,0,128, 0.3)', #Red agentes a ECO 9
               'rgba(0,0,128, 0.3)', #Red agentes a EGAS 10
               'rgba(0,0,128, 0.3)', #Corredores a Contact Center 11
               'rgba(0,0,128, 0.3)', #Clínicas a Contact Center 12
               'rgba(0,0,128, 0.3)', #Desconocido a SOL 13
               
               'rgba(249,191,59, 0.6)',     #Siebel contactos útiles 14
               'rgba(249,105,14, 0.5)',    #Siebel contactos no útiles 15 
               'rgba(192,192, 192, 0.8)',  #Siebel Pendientes o disuadidas 16
               
                'rgba(58,174,89, 0.6)',      #Contactos útiles nivel 1 retenidos 17
                'rgba(192,192, 192, 0.8)',   #Contactos útiles nivel 1 disuadidos 18
                 'rgba(249,191,59, 0.6)',     #Contactos útiles nivel 1 bajas 19
                'rgba(249,105,14, 0.7)',     #Contactos no útiles nivel 1 bajas 20
               
               
               'rgba(0,128,128,0.4)',   #Cola ECO  21
               'rgba(255,0,0, 0.3)',    #ECO bajas 22
               'rgba(192,192, 192, 0.8)',  #ECO pendientes 23
 
               'rgba(0,128,128,0.4)',    #EGAS Cola EGAS 24
               'rgba(255,0,0, 0.3)',     #EGAS bajas 25
               'rgba(58,174,89, 0.6)',   #EGAS retenidos 26
               'rgba(192,192, 192, 0.8)', #EGAS pendientes 27
 
               'rgba(0,128,128,0.4)',   #SOL cola ECO 28
               'rgba(255,0,0, 0.3)',    #SOL bajas 29
               'rgba(192,192, 192, 0.8)', #SOL pendientes 30
               
               'rgba(249,191,59, 0.7)', #Cola ECO contactos útiles 27
               'rgba(249,105,14, 0.5)', #Cola ECO contactos no útiles 28
 
               
               'rgba(249,191,59, 0.7)', #Cola EGAS contactos útiles 29
               'rgba(249,105,14, 0.5)', #Cola EGAS contactos no útiles 30
               
               'rgba(58,174,89, 0.6)', #Contactos útiles retenidos 31
                'rgba(249,191,59, 0.7)',  #Contactos útiles bajas 32
               'rgba(192,192, 192, 0.8)', #Contactos útiles disuadidos 33
               'rgba(249,105,14, 0.7)'] #Contactos no útiles bajas 34

In [13]:
source2    = [0,     0,   1,    1,     1,   3,   3,    3,  4]

target2    = [1,     2,   3,    4,     7,   5,   6,    7,  6]
value2     = [21500, 350, 4300, 17100, 100, 471, 3827, 2,  17100 ]

flow_colors2 =  [ 'rgba(0,0,128, 0.2)',      #Bajas grabadas a recuperar a  solicitud  0 
                 'rgba(255,0,0, 0.3)',    #Bajas grabadas a recuperar a sin solicitud  1
              
              'rgba(249,191,59, 0.5)',   #Solicitudes a contactos útiles 2
              'rgba(249,105,14, 0.5)',   #Solicitudes a contactos no útiles 3 
              'rgba(192,192, 192, 0.8)',  #Solicitudes a pendientes 4
              
               'rgba(58,174,89, 0.6)',    #Contactos útiles a retenidos 5
                'rgba(249,105,14, 0.5)',    #Contactos útiles bajas 6
              'rgba(192,192, 192, 0.8)',  #Contactos útiles disuadidos 7
               'rgba(249,105,14, 0.5)' ]  #Contactos no útiles a bajas 8

In [14]:


node_x_position = [ 0,          #Desconocido': 0, 
                    0.2,       #'Contact center': 1, 
                    0,       #'Oficinas Propias': 2, 
                    0,       #'OACs': 3, 
                    0,       #'Red Comercial': 4, 
                    0,          #Corredores 5
                    0,          #Clínicas b 6
                   
                   
                    0.4,       #'Siebel': 7, 
                    0.4,       #'ECO':    8, 
                    0.4 ,      #'EGAS':   9,
                    0.4,       #SOL:      10,
                   
                   
                    0.6,       #Contactos útiles nivel 1                  
                    0.6,        #Contactos no útiles nivel 1
                   
                   
                    0.6,       #'Sol Cola ECO': 11, 
                    0.6,       #'Sol Cola EGAS': 12,
                   
                    0.73,       #'Contactos Útiles': 13, 
                    0.73,       #'Contactos No Útiles': 14, 
                    
                    0.97,      #Retenidos 15
                    1,       #Bajas 16
                    0.97,  ]      #Pendientes y disuadidos 17



node_y_position = [ 0.954,        #Desconocido': 0, 
                   
                    0.025,          #'Contact center': 1, 
                    0.718,          #'Oficinas Propias': 2, 
                    0.481,         #'OACs': 3, 
                    0.054 ,         #'Red agentes': 4, 
                   
                    0.01,          #Corredores 5
                    0.034,        #Clínicas   6 
                   
                    0.01,       #'Siebel': 7, 
                    0.807,       #'ECO': 8, 
                    0.494,          #'EGAS': 9, 
                    0.985,         #SOL: 10  
                   
                    0.25,       #Contactos útiles nivel 1
                    0.01,        #Contactos no útiles nivel 1
                   
                    
                    0.87,       #'Sol Cola ECO': 11, 
                    0.53,       #'Sol Cola EGAS': 12, 
                   
                    0.8735,       #'Contactos Útiles': 13, 
                    0.763,       #'Contactos No Útiles': 14, 
                   
                    0.85,         #Retenidos 15
                    0,       #Bajas 16 
                    0.72   ] #Pendientes y disuadidos 17






In [15]:
node_x_position2 = [ 0 , #Bajas grabadas a recuperar rojo
                
                     0.3, #Solicitudes azul
                     0.3 , #Sin solicitud rojo claro
                
                     0.64 , #Contactos útiles naranja claro
                     0.64, #Contactos no útiles naranja
                
                     1 , #Retenidos verde
                     1,  #No retenidos rojo
                     1]  #Pendientes o disuadidas gris 


node_y_position2 = [ 0 , #Bajas grabadas a recuperar rojo
                
                     0, #Solicitudes azul
                     0.9 , #Sin solicitud rojo claro
                
                     0.7, #Contactos útiles naranja claro
                     0, #Contactos no útiles naranja
                
                     0.8 , #Retenidos verde
                     0,  #No retenidos rojo
                     1]  #Pendientes o disuadidas gris 

In [16]:

canal = ['Total']*39 + ['Contact center']*39

canal_position = ['Total']*20 + ['Contact center']*20

value_contact = [0 ,18500, 0, 0, 0, 500,0,  0 , 200 , 0,0 ,200,100 ,0, 13975, 7455, 70,4000, 891,9084, 7455,0,0, 
                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]





labels_final = labels * 2



node_x_position_contact = [ 0,          #Desconocido': 0, 
                            0.25,       #'Contact center': 1, 
                            0.0,       #'Oficinas Propias': 2, 
                            0,       #'OACs': 3, 
                            0,       #'Red Comercial': 4, 
                            0,          #Corredores 5
                            0,          #Clínicas b 6
                   
                   
                           0.8,       #'Siebel': 7, 
                           0,       #'ECO':    8, 
                           0 ,      #'EGAS':   9,
                           0,       #SOL:      10,
                           
                           0,      #Contactos útiles nivel 1
                           0,      #Contactos no útiles nivel 1
                        
                          0,       #'Sol Cola ECO': 11, 
                          0,       #'Sol Cola EGAS': 12,
                   
                          0,       #'Contactos Útiles': 13, 
                          0,       #'Contactos No Útiles': 14, 
                   
                          0.97,       #'Retenidos': 15, 
                          1,     #'Bajas': 16
                          0.97]      #'Pendientes y disuadidos 17






node_y_position_contact = [ 0,        #Desconocido': 0, 
                   
                    0.01,          #'Contact center': 1, 
                    0,          #'Oficinas Propias': 2, 
                    0.01,         #'OACs': 3, 
                    0.3,         #'Red agentes': 4, 
                   
                    0.5,          #Corredores 5
                    0.6,        #Clínicas   6 
                   
                    0.01,       #'Siebel': 7, 
                    0,          #'ECO': 8, 
                    0,          #'EGAS': 9, 
                    0,         #SOL: 10       
                           
                    0,      #Contactos útiles nivel 1
                    0,      #Contactos no útiles nivel 1
                        
                    
                    0,       #'Sol Cola ECO': 11, 
                    0,       #'Sol Cola EGAS': 12, 
                   
                    0,       #'Contactos Útiles': 13, 
                    0,       #'Contactos No Útiles': 14, 
                   
                    0.85,       #'Retenidos': 15, 
                    0.0,        #'Bajas': 16
                    0.72      ]      #Pendientes y disuadidos




node_y_position_final = node_y_position + node_y_position_contact

node_x_position_final = node_x_position  + node_x_position_contact

target_final = target + target
source_final = source + source
value_final = value + value_contact 


df_solicitudes = pd.DataFrame({'source': source_final,
                   'target': target_final,
                   'value': value_final,
                   'Canal': canal,})




df_node_position = pd.DataFrame({'node': labels_final,
                   'x': node_x_position_final,
                   'y': node_y_position_final,
                   'Canal': canal_position})

#df_solicitudes

#df_node_position

In [17]:
date = ['Total']*9 + ['11-2019']*9 + ['12-2019']*9



value_total2 = value2
value_201912 = [elem/2 for elem in value2]
value_201911 = [elem/2 for elem in value2]




labels2_final = labels2 * 2


target_final2 = target2 + target2 + target2
source_final2 = source2 + source2 + source2
value_final2 = value2 + value_201911 + value_201912 


df_solicitudes2 = pd.DataFrame({'source': source_final2,
                   'target': target_final2,
                   'value': value_final2,
                   'Date': date})



#df_solicitudes2



In [18]:
filter_product = widgets.Dropdown(
    options= ['Total', 'Contact center'] ,
    value='Total',
    description='Canal:',
    font_size = 7
)

In [19]:
filter_date = widgets.Dropdown(
    options= ['Total', '11-2019', '12-2019'] ,
    value='Total',
    description='Fecha envío:',
    font_size = 7
)

## Flujo grabación  y recuperación de solicitudes de baja <a class="anchor" id="1"></a>

In [20]:

#To prevent any output from a jupyter notebook cell you may start the cell with %%capture + variable


fig = go.FigureWidget(data=[go.Sankey(
    valueformat = ".0f",
    #valuesuffix = "",
    arrangement = "freeform", #snap, perpendicular, freeform, fixed
    hoverinfo = 'all',
    orientation = 'h',
    customdata = cust_data,
    
    # Define nodes
    node = dict(
      pad = 40,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  labels,
      color =  node_colors,
      x = node_x_position,
      y = node_y_position,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Total solicitudes %{value}.<extra>Retenidos %{customdata}</extra>'
    ),
    # Add links
    link = dict(
      source =  source,
      target =  target,
      value =  value,
      color = flow_colors,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Solicitudes de %{source.label} a %{target.label}: %{value}<extra></extra>'
  ))])

fig.update_layout(title_text="Solicitudes de baja (Individuales, Pymes y Colectivos abiertos)",
                  font_size=10, height = 450, width = 1000,  hoverlabel=dict(
        bgcolor="white",
        font_size=13,
        font_family="Rockwell"
    ))

#Para no mostrar el output del gráfico
clear_output(wait = True)

#fig.show()


In [21]:


fig2 = go.FigureWidget(data=[go.Sankey(
    valueformat = ".0f",
    #valuesuffix = "",
    arrangement = "freeform", #snap, perpendicular, freeform, fixed
    hoverinfo = 'all',
    orientation = 'h',
    customdata = cust_data2,
    
    # Define nodes
    node = dict(
      pad = 40,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  labels2,
      color =  node_colors2,
      x = node_x_position2,
      y = node_y_position2,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Total solicitudes %{value}.<extra>Retenidos %{customdata}</extra>'
    ),
    # Add links
    link = dict(
      source =  source2,
      target =  target2,
      value =  value2,
      color = flow_colors2,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Solicitudes de %{source.label} a %{target.label}: %{value}<extra></extra>'
  ))])

fig2.update_layout(title_text="Solicitudes de baja a recuperar",
                  font_size=10, height = 400, width = 1000,  hoverlabel=dict(
        bgcolor="white",
        font_size=13,
        font_family="Rockwell"
    ))


#Para no mostrar el output del gráfico
clear_output(wait = True)
#fig_resp2.show()


In [22]:
v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Solicitudes de  baja']),
    v.Tab(children=['Solicitudes de baja a recuperar']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=False, xl4=False, children=[
                fig
            ])
         
        ])
    ]),
     v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=False, xl4=False, children=[
                fig2
            ])
          
        ])
]) ])

Tabs(children=[Tab(children=['Solicitudes de  baja']), Tab(children=['Solicitudes de baja a recuperar']), TabI…

## Solicitudes de baja flujo interactivo <a class="anchor" id="3"></a>

In [23]:

#To prevent any output from a jupyter notebook cell you may start the cell with %%capture + variable


fig_resp = go.FigureWidget(data=[go.Sankey(
    valueformat = ".0f",
    #valuesuffix = "",
    arrangement = "freeform", #snap, perpendicular, freeform, fixed
    hoverinfo = 'all',
    orientation = 'h',
    customdata = cust_data,
    
    # Define nodes
    node = dict(
      pad = 40,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  labels,
      color =  node_colors,
      x = node_x_position,
      y = node_y_position,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Total solicitudes %{value}.<extra>Retenidos %{customdata}</extra>'
    ),
    # Add links
    link = dict(
      source =  source,
      target =  target,
      value =  value,
      color = flow_colors,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Solicitudes de %{source.label} a %{target.label}: %{value}<extra></extra>'
  ))])

fig_resp.update_layout(title_text="Solicitudes de baja (Individuales, Pymes y Colectivos abiertos)",
                  font_size=10, height = 450, width = 1000,  hoverlabel=dict(
        bgcolor="white",
        font_size=13,
        font_family="Rockwell"
    ))

#Para no mostrar el output del gráfico
clear_output(wait = True)

#fig_resp.show()

In [24]:

fig_resp2 = go.FigureWidget(data=[go.Sankey(
    valueformat = ".0f",
    #valuesuffix = "",
    arrangement = "freeform", #snap, perpendicular, freeform, fixed
    hoverinfo = 'all',
    orientation = 'h',
    customdata = cust_data2,
    
    # Define nodes
    node = dict(
      pad = 40,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  labels2,
      color =  node_colors2,
      x = node_x_position2,
      y = node_y_position2,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Total solicitudes %{value}.<extra>Retenidos %{customdata}</extra>'
    ),
    # Add links
    link = dict(
      source =  source2,
      target =  target2,
      value =  value2,
      color = flow_colors2,
#hovertemplate <extra></extra> muestra la información de la caja secundaria
      hovertemplate = 'Solicitudes de %{source.label} a %{target.label}: %{value}<extra></extra>'
  ))])

fig_resp2.update_layout(title_text="Solicitudes de baja a recuperar",
                  font_size=10, height = 400, width = 1000,  hoverlabel=dict(
        bgcolor="white",
        font_size=13,
        font_family="Rockwell"
    ))


#Para no mostrar el output del gráfico
clear_output(wait = True)
#fig_resp2.show()

In [25]:
def response_fig1(change):
          
     
        with fig_resp.batch_update():
                fig_resp.data[0].link['source'] = df_solicitudes[df_solicitudes['Canal'] == str(filter_product.value) ]['source']
                fig_resp.data[0].link['target'] = df_solicitudes[df_solicitudes['Canal'] == str(filter_product.value) ]['target']
                fig_resp.data[0].link['value'] =  df_solicitudes[df_solicitudes['Canal'] == str(filter_product.value) ]['value']
                fig_resp.data[0].node['x'] = df_node_position[df_node_position['Canal'] == str(filter_product.value) ]['x']
               
                #fig_rep.data[0].node['y'] = df_node_position[df_node_position['Canal'] == str(filter_product.value) ]['y']

def response_fig2(change):
             
        with fig_resp2.batch_update():
                fig_resp2.data[0].link['source'] = df_solicitudes2[df_solicitudes2['Date'] == str(filter_date.value) ]['source']
                fig_resp2.data[0].link['target'] = df_solicitudes2[df_solicitudes2['Date'] == str(filter_date.value) ]['target']
                fig_resp2.data[0].link['value'] =  df_solicitudes2[df_solicitudes2['Date'] == str(filter_date.value) ]['value']
                               
                
                
                
filter_product.observe(response_fig1, names="value")

filter_date.observe(response_fig2, names="value")

HBox_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='25%')

title_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%',
                    height = '8%')

container1 = widgets.HBox([filter_product], layout = HBox_layout)
container2 = widgets.HBox([filter_date], layout = HBox_layout)

title = widgets.HTML(value = '<p style="font-size:160%;text-align:center;">Solicitudes de baja canal asegurador</p>', layout = title_layout)

#title = widgets.HTML(value = '<b>Solicitudes de baja canal asegurador</b>', style = b{font_size: 160%;}, layout = title_layout)

VBox_layout = Layout( height='1050px', widht = '1000px')

#widgets.VBox([title,container1,
#              fig_resp, container2, fig_resp2], layout = VBox_layout, title_widget = 'TEST', box_style = 'success')


box_1 = widgets.VBox([container1,
           fig_resp], layout = VBox_layout, title_widget = 'TEST', box_style = 'success')


box_2 = widgets.VBox([container2,
           fig_resp2], layout = VBox_layout, title_widget = 'TEST', box_style = 'success')


clear_output(wait = True)


In [26]:
v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Solicitudes de  baja']),
    v.Tab(children=['Solicitudes de baja a recuperar']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=False, xl4=False, children=[
                box_1
            ])
         
        ])
    ]),
     v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=False, xl4=False, children=[
                box_2
            ])
          
        ])
]) ])

Tabs(children=[Tab(children=['Solicitudes de  baja']), Tab(children=['Solicitudes de baja a recuperar']), TabI…